In [ ]:
!pip install -q sentence-transformers faiss-cpu transformers accelerate bitsandbytes flask pyngrok requests


In [ ]:
import os
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [ ]:
KNOWLEDGE_BASE = [
    {
        "id": "doc01",
        "title": "Shoplite User Registration Process",
        "content": """To create a Shoplite account, users visit the registration page and provide an email address, password, and a display name. After submitting, a verification email is sent; users must verify within 24 hours or the registration token expires. Shoplite supports two primary account types: Buyer and Seller. Buyer accounts are free and enable browsing, buying, saving favorites, writing reviews, and tracking orders. Seller accounts require additional business information: company name, registration ID/tax ID, bank or payout information, and a short description of the seller's product categories.

Seller verification requires uploading supporting documents (business license or tax ID) and typically takes 2–3 business days for manual review. During this time, sellers can complete their storefront setup but cannot create public product listings until verification completes. Password policy: minimum 10 characters, at least one uppercase letter, one digit, and one special character. Two-factor authentication (2FA) via SMS or authenticator apps is optional but recommended for sellers. For account recovery, users can request a recovery link sent to the registered email; the link expires after 1 hour.

Word Count: 261"""
    },
    {
        "id": "doc02",
        "title": "Shoplite Shopping Cart Features",
        "content": """The Shoplite shopping cart allows users to add multiple items across different categories, save the cart for later, and apply discount codes before checkout. The cart persists across devices when logged in. Items are automatically reserved for 15 minutes once added to the cart, after which they may be released if checkout is not completed. Users can increase or decrease item quantities, remove items, and view updated totals in real time. Shipping costs and estimated delivery times are calculated dynamically based on the user's address. If an item goes out of stock while in the cart, the user is notified immediately.

Shoplite supports guest carts, but these are only stored in browser cookies and expire after 24 hours or if the cache is cleared. Logged-in users’ carts are stored on the server side and can be retrieved on any device. For promotions, Shoplite allows stacking one site-wide discount code with one seller-specific discount code. Loyalty points are applied automatically at checkout.

Word Count: 236"""
    },
    {
        "id": "doc03",
        "title": "Shoplite Checkout & Payment",
        "content": """Shoplite checkout is a three-step process: (1) Address confirmation, (2) Payment method selection, and (3) Review & Pay. Accepted payment methods include major credit cards (Visa, MasterCard, AmEx), PayPal, and Shoplite Pay Later (installments). Sensitive payment data is processed through PCI-DSS Level 1 compliant gateways. Shoplite does not store raw card details; instead, tokens are stored for future payments.

Fraud detection runs at checkout using machine learning signals such as IP geolocation mismatch, unusual device fingerprints, and transaction velocity. If flagged, the system requests additional verification or places the order on hold for manual review. Discounts and loyalty points are applied automatically at payment review. Receipts are emailed instantly, and order status changes are updated in the user dashboard.

Word Count: 247"""
    },
    {
        "id": "doc04",
        "title": "Shoplite Seller Onboarding",
        "content": """New sellers complete a guided onboarding wizard. Required steps: (1) Submit business details, (2) Verify identity and banking, (3) Upload at least one product listing, and (4) Agree to Shoplite Seller Terms. A seller dashboard tutorial introduces analytics, inventory management, and promotions. Once approved, sellers receive a “Verified” badge.

Seller payouts are weekly by default, with an option to request faster payouts (2 days) at a 1% processing fee. Disputes and returns are handled via the dashboard. Sellers are responsible for updating inventory in real-time. Repeated stockouts may lower seller rating.

Word Count: 217"""
    },
    {
        "id": "doc05",
        "title": "Shoplite Shipping & Delivery",
        "content": """Shoplite integrates with DHL, FedEx, UPS, and national postal services. Delivery estimates are shown at checkout. Standard delivery is 3–5 days; express is 1–2 days. Orders above $100 qualify for free standard shipping. Tracking numbers are automatically sent to users and updated in their dashboard. Buyers may select delivery to home, work, or parcel lockers.

Shoplite’s SLA guarantees on-time delivery for express orders. If delayed, buyers receive a partial refund of shipping fees. Sellers must ship within 48 hours of order placement; late shipments affect seller ratings. Shoplite also provides buyer protection for lost or damaged goods.

Word Count: 214"""
    },
    {
        "id": "doc06",
        "title": "Shoplite Returns & Refunds",
        "content": """Buyers may initiate returns within 14 days of delivery. Eligible items must be unused, with tags, and in original packaging. Exceptions: perishable goods, digital downloads, and personalized items. Return shipping is buyer-paid unless the product is defective or incorrect. Refunds are issued within 5–7 days after the item is received and inspected by the seller.

Shoplite mediates disputes if sellers fail to honor return policies. Escalations are reviewed within 72 hours. Partial refunds may apply if items are returned with missing accessories. Refunds are always sent to the original payment method.

Word Count: 198"""
    },
    {
        "id": "doc07",
        "title": "Shoplite Buyer Protection",
        "content": """Shoplite guarantees buyers a refund if items do not arrive, arrive damaged, or are significantly not as described. Claims must be filed within 30 days of delivery estimate. Shoplite covers the full order value including shipping. Buyers are required to upload photos for damaged-item claims.

Sellers who accumulate frequent claims may face penalties including reduced visibility or suspension. Buyer protection does not cover buyer’s remorse or unauthorized use of an account. Fraudulent claims are investigated and may result in buyer suspension.

Word Count: 185"""
    },
    {
        "id": "doc08",
        "title": "Shoplite Notifications System",
        "content": """Shoplite sends notifications via email, push (mobile/web), and in-dashboard alerts. Buyers receive notifications for order updates, promotions, and cart reminders. Sellers are notified about orders, low stock, disputes, and payout confirmations. Critical system events (e.g., failed payments) trigger immediate notifications. Users may manage preferences under Account Settings > Notifications.

Shoplite ensures delivery reliability by retrying failed notifications. Push notifications require explicit opt-in. Marketing emails comply with GDPR/CCPA, with easy unsubscribe options. Notification logs are retained for 90 days.

Word Count: 191"""
    },
    {
        "id": "doc09",
        "title": "Shoplite Loyalty Program",
        "content": """Buyers earn 1 point per $1 spent. Points can be redeemed for discounts: 100 points = $5. Points expire after 12 months if unused. Bonus points are offered during seasonal campaigns. Points are applied automatically at checkout unless disabled by the buyer.

Sellers can fund extra loyalty rewards for their stores, boosting retention. The program dashboard shows users their points balance, redemption history, and expiry reminders. Fraudulent point accumulation (e.g., self-purchases) is monitored and penalized.

Word Count: 182"""
    },
    {
        "id": "doc10",
        "title": "Shoplite Customer Support",
        "content": """Shoplite support is available 24/7 via chat, email, and phone. AI chatbots handle FAQs, with escalation to human agents for complex issues. Average response time: chat (under 1 min), email (within 12 hours), phone (under 5 min wait). Support covers account issues, order disputes, refunds, and technical troubleshooting.

Users can also access a self-service Help Center with guides and video tutorials. Premium sellers have access to priority support with a dedicated account manager.

Word Count: 182"""
    },
    {
        "id": "doc11",
        "title": "Shoplite Analytics for Sellers",
        "content": """The seller dashboard provides sales reports, conversion rates, traffic sources, and customer demographics. Sellers can filter by date range, product, or campaign. Advanced analytics include heatmaps of user activity and prediction of inventory needs using machine learning.

Reports are exportable to CSV or Excel. Real-time dashboards update every 5 minutes. Premium sellers access A/B testing tools for promotions and storefront design.

Word Count: 178"""
    },
    {
        "id": "doc12",
        "title": "Shoplite Security Policies",
        "content": """Shoplite enforces HTTPS across all services, encrypts passwords with bcrypt, and supports 2FA. Access tokens expire after 1 hour; refresh tokens after 30 days. Suspicious logins trigger alerts and temporary account locks. Shoplite undergoes quarterly penetration testing and annual SOC 2 audits.

Developers follow a secure SDLC with code reviews and dependency scanning. Security incidents are disclosed within 72 hours as per compliance obligations. Users may report vulnerabilities via a public bug bounty program.

Word Count: 187"""
    },
    {
        "id": "doc13",
        "title": "Shoplite Promotions & Discounts",
        "content": """Shoplite supports site-wide campaigns, seller-specific discounts, and personalized coupons. Discounts may be percentage-based, fixed-amount, or “buy X get Y.” Coupons have customizable expiry dates, usage limits, and audience targeting (e.g., new buyers only).

Sellers can stack one discount with site-wide campaigns. Shoplite tracks redemption metrics for sellers. Abusive coupon use is flagged automatically and blocked.

Word Count: 178"""
    },
    {
        "id": "doc14",
        "title": "Shoplite Digital Products Policy",
        "content": """In addition to physical goods, Shoplite allows digital product sales (e-books, software, music). Files must be uploaded to Shoplite’s secure storage; max size per file: 5GB. Buyers receive download links after payment, valid for 48 hours and limited to 3 downloads.

Refunds for digital goods are only granted if the file is corrupted or inaccessible. Sellers must ensure compliance with intellectual property rights. Anti-piracy watermarks are applied to files.

Word Count: 177"""
    },
    {
        "id": "doc15",
        "title": "Shoplite Account Suspension Policy",
        "content": """Accounts may be suspended for policy violations such as fraud, counterfeit sales, chargebacks, or abusive behavior. Suspended users receive an email with reasons and appeal instructions. Appeals must be submitted within 14 days. During suspension, sellers cannot withdraw funds, and buyers lose loyalty point access.

Permanent bans occur for repeat violations or severe fraud. Shoplite may cooperate with law enforcement where applicable.

Word Count: 176"""
    }
]

print(f"Loaded {len(KNOWLEDGE_BASE)} docs")


In [ ]:
EMB_MODEL_NAME = "all-mpnet-base-v2"  # good general embedding model
embedder = SentenceTransformer(EMB_MODEL_NAME)

docs_texts = [d["content"] for d in KNOWLEDGE_BASE]
doc_ids = [d["id"] for d in KNOWLEDGE_BASE]

# Create embeddings
embeddings = embedder.encode(docs_texts, show_progress_bar=True, convert_to_numpy=True)

# Normalize for cosine similarity with inner product
faiss.normalize_L2(embeddings)
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)
print("FAISS index built. ndim:", dim, "n_docs:", index.ntotal)

def retrieve(query, k=4):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D,I = index.search(q_emb, k)
    results = []
    for idx, score in zip(I[0], D[0]):
        if idx < 0:
            continue
        doc = KNOWLEDGE_BASE[idx]
        results.append({
            "id": doc["id"],
            "title": doc["title"],
            "content": doc["content"],
            "score": float(score)
        })
    return results


In [ ]:
PREFERRED_MODEL = "meta-llama/Llama-3-8b"
FALLBACK_MODEL = "tiiuae/falcon-7b-instruct"

hf_token = input("Enter Hugging Face token if needed (press Enter to skip): ").strip()

def load_model(model_id):
    print("Attempting to load model:", model_id)
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token or None)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            load_in_8bit=True,
            use_auth_token=hf_token or None
        )
        gen = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
        print("Loaded model:", model_id)
        return tokenizer, model, gen
    except Exception as e:
        print("Model load failed:", e)
        return None, None, None

tokenizer, model, generator = load_model(PREFERRED_MODEL)
if generator is None:
    print("Falling back to:", FALLBACK_MODEL)
    tokenizer, model, generator = load_model(FALLBACK_MODEL)
    if generator is None:
        raise RuntimeError("No usable model loaded. Try a smaller model or update Colab runtime to GPU.")

def format_prompt_with_docs(question, retrieved_docs):
    docs_snippets = "\n\n".join([f"### {d['title']}\n{d['content'][:1500]}" for d in retrieved_docs])
    prompt = f"""You are a Shoplite assistant. Use only the documents provided below to answer the user's question. Cite the document titles in the Sources section.

DOCUMENTS:
{docs_snippets}

QUESTION:
{question}

Answer concisely. At the end include a Sources line listing which documents you used.
"""
    return prompt

def generate_answer(prompt, max_tokens=256):
    # We call generator with a conservative max_new_tokens
    out = generator(prompt, max_new_tokens=max_tokens, do_sample=False)[0]["generated_text"]
    # sometimes the model echoes the prompt; attempt to strip prompt if it appears
    if out.startswith(prompt):
        return out[len(prompt):].strip()
    return out.strip()


In [ ]:
# Small utility to shorten long doc snippets when building prompts (keeps prompt size reasonable)
def short_snippet(text, max_chars=1200):
    if len(text) <= max_chars:
        return text
    return text[:max_chars].rsplit('\n', 1)[0] + "\n\n...[truncated]"

# Example: use short_snippet when you need to reduce prompt size manually


In [ ]:
app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"})

@app.route('/ping', methods=['POST'])
def ping():
    data = request.json or {}
    user_prompt = data.get("prompt", "Hello")
    # Force concise, single-sentence test response
    wrapped = f"You are a Shoplite assistant. Reply in one short sentence confirming you are online.\nUser: {user_prompt}\nAssistant:"
    answer = generate_answer(wrapped)
    return jsonify({"answer": answer})

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json or {}
    question = data.get("question") or data.get("q") or ""
    if not question:
        return jsonify({"error": "No question provided"}), 400
    retrieved = retrieve(question, k=4)
    # Use short_snippet to avoid overly long prompts
    docs_snippets = "\n\n".join([f"### {d['title']}\n{short_snippet(d['content'])}" for d in retrieved])
    wrapped = f"You are a helpful Shoplite support assistant. Answer concisely and only using the information in the documents below. If the answer is not in the documents, say you don't know and suggest next steps.\n\nDOCUMENTS:\n{docs_snippets}\n\nQUESTION: {question}\n\nAssistant:"
    answer = generate_answer(wrapped)
    return jsonify({
        "answer": answer,
        "sources": [r["title"] for r in retrieved]
    })


In [ ]:
ngrok_token = input("Enter your ngrok token (will not be saved): ").strip()
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

from threading import Thread
def run():
    app.run(host="0.0.0.0", port=5000)
t = Thread(target=run)
t.start()
print("Flask started on port 5000.")


In [ ]:
import requests
BASE_URL = public_url

print("---- Health Test ----")
try:
    print(requests.get(BASE_URL + "/health").json())
except Exception as e:
    print("Health check failed:", e)

print("\n---- Ping Test ----")
try:
    ping_resp = requests.post(BASE_URL + "/ping", json={"prompt": "Hello Shoplite!"}, timeout=120).json()
    print(ping_resp)
except Exception as e:
    print("Ping failed:", e)

print("\n---- Chat Test ----")
try:
    payload = {"question": "How long does seller verification take?"}
    chat_resp = requests.post(BASE_URL + "/chat", json=payload, timeout=120).json()
    print(chat_resp)
except Exception as e:
    print("Chat failed:", e)
